In [1]:
from pyspark.sql import SparkSession




spark = SparkSession.builder \
.appName ("Spark_Cache_Demo_RDD") \
.getOrCreate()


25/02/06 07:46:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

In [3]:
!hadoop fs -ls /data/

Found 13 items
-rw-r--r--   2 root hadoop  343317147 2025-02-03 11:53 /data/customers.csv
-rw-r--r--   2 root hadoop       5488 2025-02-03 11:53 /data/customers_100.csv
-rw-r--r--   2 root hadoop   10528211 2025-02-03 11:53 /data/customers_10mb.csv
-rw-r--r--   2 root hadoop    1060750 2025-02-03 11:53 /data/customers_1mb.csv
-rw-r--r--   2 root hadoop  570783961 2025-02-03 11:53 /data/customers_500mb.csv
-rw-r--r--   2 root hadoop        351 2025-02-04 05:35 /data/customers_with_errors.csv
-rw-r--r--   2 root hadoop         23 2025-02-04 14:53 /data/date.csv
-rw-r--r--   2 root hadoop        209 2025-02-05 06:28 /data/dates_data.csv
drwxr-xr-x   - root hadoop          0 2025-02-05 11:43 /data/external_data
drwxr-xr-x   - root hadoop          0 2025-02-04 07:53 /data/write_output.csv
drwxr-xr-x   - root hadoop          0 2025-02-04 07:56 /data/write_output_1_part
drwxr-xr-x   - root hadoop          0 2025-02-04 07:59 /data/write_output_1_part_2
drwxr-xr-x   - root hadoop          0 202

In [8]:
customers_rdd = spark.sparkContext.textFile('/data/customers_1mb.csv')

In [9]:
customers_filtered = customers_rdd.filter(lambda row:'Mumbai' in row)
customers_mapped = customers_filtered.map(lambda row: (row.split(',')[0],1))
customers_reduced = customers_mapped.reduceByKey(lambda x,y:x+y)

In [10]:
customers_reduced.count()

# 1st run --> 4 sec

2142

In [13]:
customers_reduced.getNumPartitions()

2

In [12]:
# 2nd Run 
 
customers_reduced.count()

# Visibly Faster


2142

In [ ]:
# 3rd Run also faster

customers_reduced.count()

In [14]:
customers_reduced.cache() # Caching Is LAZY

PythonRDD[15] at RDD at PythonRDD.scala:53

In [15]:
customers_reduced.count()

2142

In [16]:
customers_reduced.count()

2142

In [18]:
customers_reduced.unpersist() # To Uncache

PythonRDD[15] at RDD at PythonRDD.scala:53

In [19]:
spark.stop()